In [1]:
import pandas as pd

In [2]:
historical_stock_prices = pd.read_csv('daily-historical-stock-prices-1970-2018/historical_stock_prices.csv')

In [3]:
historical_stocks = pd.read_csv('daily-historical-stock-prices-1970-2018/historical_stocks.csv')

In [4]:
historical_stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [5]:
historical_stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [6]:
combined_stocks_df = pd.merge(historical_stock_prices, historical_stocks, on='ticker', how='left')

In [17]:
sorted_date = combined_stocks_df.sort_values(by='date', ascending=False)

In [18]:
sorted_date.head()

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
20973888,NZF,14.600000,14.690000,14.690000,14.590000,14.690000,180900,2018-08-24,NYSE,NUVEEN MUNICIPAL CREDIT INCOME FUND,NaN,NaN
18244951,LNN,91.269997,92.400002,92.400002,91.059998,92.580002,23300,2018-08-24,NYSE,LINDSAY CORPORATION,CAPITAL GOODS,INDUSTRIAL MACHINERY/COMPONENTS
9333663,EGF,13.090000,13.010000,13.010000,13.010000,13.090000,200,2018-08-24,NYSE,"BLACKROCK ENHANCED GOVERNMENT FUND, INC",NaN,NaN
9332474,MPWR,146.389999,147.240005,147.240005,146.080002,147.600006,190300,2018-08-24,NASDAQ,"MONOLITHIC POWER SYSTEMS, INC.",TECHNOLOGY,SEMICONDUCTORS
18233902,SGEN,75.599998,74.589996,74.589996,73.889999,75.839996,398200,2018-08-24,NASDAQ,"SEATTLE GENETICS, INC.",HEALTH CARE,BIOTECHNOLOGY: BIOLOGICAL PRODUCTS (NO DIAGNOS...


In [19]:
grouped_stocks_ticker = combined_stocks_df.groupby('ticker')

In [23]:
max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']

In [31]:
currently_trading_ticker = max_date_df = combined_stocks_df.loc[combined_stocks_df['date'] == '2018-08-24']['ticker']

In [33]:
currently_trading_ticker = currently_trading_ticker.values

In [44]:
filtered_stocks = pd.DataFrame(columns=('ticker', 'open', 'close', 'adj_close', 'low', 'high', 'volume', 'date', 'exchange', 'name', 'sector', 'industry'))

In [45]:
filtered_stocks

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry


In [ ]:
for index, row in combined_stocks_df.iterrows():
    if row[0] in currently_trading_ticker:
        filtered_stocks.append(combined_stocks_df.iloc[index,:])

In [48]:
currently_trading_df = combined_stocks_df[combined_stocks_df['ticker'].isin(currently_trading_ticker)]

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,AHH,11.50,11.580000,8.493155,11.25,11.680000,4633900,2013-05-08,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
1,AHH,11.66,11.550000,8.471151,11.50,11.660000,275800,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2,AHH,11.55,11.600000,8.507822,11.50,11.600000,277100,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
3,AHH,11.63,11.650000,8.544494,11.55,11.650000,147400,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
4,AHH,11.60,11.530000,8.456484,11.50,11.600000,184100,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
5,AHH,11.60,11.600000,8.507822,11.54,11.600000,76800,2013-05-15,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
6,AHH,11.62,11.740000,8.610501,11.54,11.740000,170300,2013-05-16,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
7,AHH,11.70,11.760000,8.625172,11.70,11.850000,305400,2013-05-17,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
8,AHH,11.76,11.730000,8.603169,11.63,11.830000,46800,2013-05-20,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
9,AHH,11.76,11.830000,8.676512,11.61,11.840000,77000,2013-05-21,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
